In [1]:
import json

In [2]:
#download portland geojson
!wget -q -O 'pdx_data.geojson' https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson

In [6]:
with open('pdx_data.geojson') as f:
    pdx_data = json.load(f)

In [10]:
pdx_data['features'][0]

{'type': 'Feature',
 'properties': {'OBJECTID': 1,
  'NAME': 'CATHEDRAL PARK',
  'COMMPLAN': ' ',
  'SHARED': ' ',
  'COALIT': 'NPNS',
  'HORZ_VERT': 'HORZ',
  'MAPLABEL': 'Cathedral Park',
  'ID': 31,
  'Shape_Length': 11434.254777015894,
  'Shape_Area': 5424297.812272152},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-122.74774080646131, 45.57641768898887],
    [-122.74500149910529, 45.57857491566089],
    [-122.74269522193698, 45.58047343916037],
    [-122.74104347614936, 45.581833404649124],
    [-122.73855205350608, 45.583948080459194],
    [-122.73985832520177, 45.584275214997184],
    [-122.7395075097271, 45.58494001451983],
    [-122.74359031676082, 45.585914885762904],
    [-122.74467513128107, 45.58617357843923],
    [-122.74474030674988, 45.5860431820492],
    [-122.74654134894452, 45.58662638095421],
    [-122.74611755343784, 45.58727221988719],
    [-122.74867056996688, 45.58809397668034],
    [-122.74900079695354, 45.58755267421378],
    [-122.75302440680665, 45.5